In [2]:
!python --version

Python 3.11.2


In [3]:
!pip list

Package           Version
----------------- -------
asttokens         2.2.1
backcall          0.2.0
colorama          0.4.6
comm              0.1.3
debugpy           1.6.7
decorator         5.1.1
executing         1.2.0
ipykernel         6.23.1
ipython           8.14.0
jedi              0.18.2
jupyter_client    8.2.0
jupyter_core      5.3.0
matplotlib-inline 0.1.6
nest-asyncio      1.5.6
packaging         23.1
parso             0.8.3
pickleshare       0.7.5
pip               22.3.1
platformdirs      3.5.1
prompt-toolkit    3.0.38
psutil            5.9.5
pure-eval         0.2.2
Pygments          2.15.1
python-dateutil   2.8.2
pywin32           306
pyzmq             25.1.0
setuptools        65.5.0
six               1.16.0
stack-data        0.6.2
tornado           6.3.2
traitlets         5.9.0
wcwidth           0.2.6



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
!pip install pytest 
!pip install ipytest
!pip install ../..[pandas] # install wheel from relative path 


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
!python --version
!pip list

Python 3.9.13
Package                  Version
------------------------ ---------
anyio                    3.6.2
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.2.1
attrs                    22.2.0
backcall                 0.2.0
beautifulsoup4           4.12.0
binaryornot              0.4.4
bleach                   6.0.0
certifi                  2022.12.7
cffi                     1.15.1
chardet                  5.1.0
charset-normalizer       3.1.0
click                    8.1.3
colorama                 0.4.6
coloredlogs              15.0.1
comm                     0.1.2
contourpy                1.0.7
cookiecutter             2.1.1
cycler                   0.11.0
debugpy                  1.6.6
decorator                5.1.1
defusedxml               0.7.1
exceptiongroup           1.1.1
executing                1.2.0
fastjsonschema           2.16.3
flowrunner               0.2.2
fonttools                4.39.2
fqdn  


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
import pytest 
import ipytest
import flowrunner # make sure we can import our wheel correctly 

ipytest.autoconfig()

In [51]:
import pandas as pd

from flowrunner import BaseFlow, end, start, step


class ExamplePandas(BaseFlow):
    @start
    @step(next=["transformation_function_1", "transformation_function_2"])
    def create_data(self):
        """
        This method we create the dataset we are going use. In real use cases,
        you'll have to read from a source (csv, parquet, etc)

        For this example we create two dataframes for students ranked by marked scored
        for when they attempted the example on 1st January 2023 and 12th March 2023

        After creating the dataset we pass it to the next methods

        - transformation_function_1
        - transformation_function_2
        """
        data1 = {"Name": ["Hermione", "Harry", "Ron"], "marks": [100, 85, 75]}

        data2 = {"Name": ["Hermione", "Ron", "Harry"], "marks": [100, 90, 80]}

        df1 = pd.DataFrame(data1, index=["rank1", "rank2", "rank3"])

        df2 = pd.DataFrame(data2, index=["rank1", "rank2", "rank3"])

        self.input_data_1 = df1
        self.input_data_2 = df2

    @step(next=["append_data"])
    def transformation_function_1(self):
        """
        Here we add a snapshot_date to the input dataframe of 2023-03-12
        """
        transformed_df = self.input_data_1
        transformed_df.insert(1, "snapshot_date", "2023-03-12")
        self.transformed_df_1 = transformed_df

    @step(next=["append_data"])
    def transformation_function_2(self):
        """
        Here we add a snapshot_date to the input dataframe of 2023-01-01
        """
        transformed_df = self.input_data_2
        transformed_df.insert(1, "snapshot_date", "2023-01-01")
        self.transformed_df_2 = transformed_df

    @step(next=["show_data"])
    def append_data(self):
        """
        Here we append the two dataframe together
        """
        self.final_df = pd.concat([self.transformed_df_1, self.transformed_df_2])

    @end
    @step
    def show_data(self):
        """
        Here we show the new final dataframe of aggregated data. However in real use cases. It would
        be more likely to write the data to some final layer/format
        """
        print(self.final_df)
        return self.final_df


In [52]:
%%ipytest

example_pandas = ExamplePandas()

def test_run():
    example_pandas.run()

def test_show():
    example_pandas.show()
    
def test_validate():
    example_pandas.validate()
    
def test_display():
    example_pandas.display()

======================================= test session starts =======================================
platform win32 -- Python 3.9.13, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\prith\Documents\VSCode_Workspace\Repos\flowrunner
configfile: pyproject.toml
plugins: anyio-3.6.2
collected 0 items

====================================== no tests ran in 0.00s ======================================


In [53]:
%%ipytest

ipytest.run()


======================================= test session starts =======================================
platform win32 -- Python 3.9.13, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\prith\Documents\VSCode_Workspace\Repos\flowrunner
configfile: pyproject.toml
plugins: anyio-3.6.2
collected 0 items

====================================== no tests ran in 0.01s ======================================


<ExitCode.NO_TESTS_COLLECTED: 5>

======================================= test session starts =======================================
platform win32 -- Python 3.9.13, pytest-7.3.1, pluggy-1.0.0
rootdir: c:\Users\prith\Documents\VSCode_Workspace\Repos\flowrunner
configfile: pyproject.toml
plugins: anyio-3.6.2
collected 0 items

====================================== no tests ran in 0.00s ======================================


In [54]:
!pytest .

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-7.3.1, pluggy-1.0.0 -- c:\Users\prith\Documents\VSCode_Workspace\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\prith\Documents\VSCode_Workspace\Repos\flowrunner
configfile: pyproject.toml
plugins: anyio-3.6.2
collecting ... collected 0 items

============================ no tests ran in 0.02s ============================
